In [1]:
# --------------------------------
import warnings
warnings.filterwarnings('ignore')
# --------------------------------

import pandas as pd
import numpy as np
import matplotlib.pyplot  as plt
import seaborn as sns

%matplotlib inline
#loading in datasets
def load_data():
  members = pd.read_csv('membership_info.csv')
  zip_data = pd.read_csv('zip_info.csv')
  volume = pd.read_csv('volume_info.csv')
  products = pd.read_csv('product_usage.csv')
  volume.fillna(0, inplace=True)
  products.fillna(0, inplace=True)
  members['number_of_locations'] = members['number_of_locations'].fillna(1)
  return volume, products, members, zip_data

In [2]:
load_data()

(          nmg_id   volume        date
 0         8310.0   3296.0  2012-12-01
 1         7955.0   1569.0  2012-12-01
 2        20044.0  25974.0  2012-12-01
 3         5235.0   8035.0  2012-12-01
 4       113248.0   6009.0  2012-12-01
 ...          ...      ...         ...
 296855    1686.0    899.0  2019-02-01
 296856    1270.0    172.0  2019-02-01
 296857    1510.0   2009.0  2019-02-01
 296858    4555.0    210.0  2019-02-01
 296859    2232.0   1419.0  2019-02-01
 
 [296860 rows x 3 columns],
               date  nmg_id  credit_card_processing_total  \
 0       2016-03-01    5557                           0.0   
 1       2021-07-01    7499                           0.0   
 2       2021-10-01    2191                           0.0   
 3       2019-04-01  104900                           0.0   
 4       2018-12-01    1495                           0.0   
 ...            ...     ...                           ...   
 181715  2023-05-01  104976                           0.0   
 181716  2022-

(          nmg_id   volume        date
 0         8310.0   3296.0  2012-12-01
 1         7955.0   1569.0  2012-12-01
 2        20044.0  25974.0  2012-12-01
 3         5235.0   8035.0  2012-12-01
 4       113248.0   6009.0  2012-12-01
 ...          ...      ...         ...
 296855    1686.0    899.0  2019-02-01
 296856    1270.0    172.0  2019-02-01
 296857    1510.0   2009.0  2019-02-01
 296858    4555.0    210.0  2019-02-01
 296859    2232.0   1419.0  2019-02-01
 
 [296860 rows x 3 columns],
               date  nmg_id  credit_card_processing_total  \
 0       2016-03-01    5557                           0.0   
 1       2021-07-01    7499                           0.0   
 2       2021-10-01    2191                           0.0   
 3       2019-04-01  104900                           0.0   
 4       2018-12-01    1495                           0.0   
 ...            ...     ...                           ...   
 181715  2023-05-01  104976                           0.0   
 181716  2022-

### Data Preparation Function

In [3]:
def prepare_function_data(volume, products, input_date_str):
    volume['date'] = pd.to_datetime(volume['date'])
    products['date'] = pd.to_datetime(products['date'])
    input_date = pd.to_datetime(input_date_str)

    # start dates for trailing 12 month and 12 months previous
    t12_start = input_date - pd.DateOffset(months=12)
    p12_start = t12_start - pd.DateOffset(months=12)

    volume_t12 = volume.query("date > @t12_start & date <= @input_date").groupby('nmg_id')['volume'].sum().reset_index(name='t12_volume')
    volume_p12 = volume.query("date > @p12_start & date <= @t12_start").groupby('nmg_id')['volume'].sum().reset_index(name='p12_volume')

    # filter for T12 in products and sum numeric columns
    products_t12 = products.query("date > @t12_start & date <= @input_date")
    products_t12_sum = products_t12.groupby('nmg_id').agg({col: 'sum' for col in products_t12 if col not in ['date', 'nmg_id']}).reset_index()

    final_df = volume_t12.set_index('nmg_id').join(volume_p12.set_index('nmg_id'), on='nmg_id', how='outer', rsuffix='_p12')
    final_df = final_df.join(products_t12_sum.set_index('nmg_id'), on='nmg_id', how='outer')

    #  volume growth calculation
    final_df['t12_volume_growth'] = (final_df['t12_volume'] - final_df.get('p12_volume', 0))

    # filter out useless data points that will hurt knn model
    final_df_filtered = final_df[
        (final_df['t12_volume'] > 0) &
        (final_df['p12_volume'] > 0) &
        (final_df['t12_volume_growth'] / final_df['p12_volume'] <= 0.5)
    ].reset_index()

    # columns to include 't12_' prefix as necessary
    for col in ['credit_card_processing_total', 'inventory_finance_total', 'lease_to_own_total', 'product_protection_total', 'retail_credit_total']:
        if col in final_df_filtered.columns:
            final_df_filtered.rename(columns={col: f't12_{col}'}, inplace=True)
    final_df_filtered['volume_growth']= (final_df_filtered['t12_volume']-final_df_filtered['p12_volume'])/final_df_filtered['p12_volume']
    final_df_filtered = final_df_filtered.fillna(0)
    final_df_filtered = final_df_filtered.drop('t12_volume_growth',axis=1)
    # joining everything
    final_with_members = final_df_filtered.reset_index().merge(members, on='nmg_id', how='outer')

    members['zip_code'] = members['zip_code'].astype(str)
    zip_data['geo_id'] = zip_data['geo_id'].astype(str)

    final_with_zip = final_with_members.merge(zip_data, left_on='zip_code', right_on='geo_id', how='outer')

    final_with_zip = final_with_zip.fillna(0)
    final_with_zip = final_with_zip.drop(['index', 'level_0', 'geo_id'], axis = 1,errors='ignore')
    def categorize_locations(value):
      if value == 1:
          return "Single Retailer"
      elif value < 5:
          return "Medium-sized Retailer"
      else:
         return "Large Retailer"

    final_with_zip['number_of_locations'] = final_with_zip['number_of_locations'].apply(categorize_locations)

    return final_with_zip


## Data Prep Function Usage

In [23]:
volume, products, members, zip_data=load_data()

In [24]:
volume['nmg_id']=volume['nmg_id'].astype('i')

In [25]:
function_data = prepare_function_data(volume, products, '2024-01-01')

In [29]:
function_data['nmg_id']=function_data['nmg_id'].astype('i')

In [32]:
function_data.head()

,nmg_id,t12_volume,p12_volume,t12_credit_card_processing_total,t12_inventory_finance_total,t12_lease_to_own_total,t12_product_protection_total,t12_retail_credit_total,volume_growth,company_name,industry_name,number_of_locations,zip_code,median_income,households,poverty_rate,total_pop,vacant_housing_rate,income_per_capita
0,1000,171197.83,277735.60,0.00,1027568.42,0.0,195782.19,96131.18,-0.383594,A-1 HOME APPLIANCE CENTER,Appliances,Medium-sized Retailer,70058,49551.0,13844.0,0.514230,40191.0,0.081841,27419.0
1,1002,36097.56,89740.93,0.00,2050776.48,0.0,66616.09,178371.82,-0.597758,BOB WALLACE APPLIANCE,Appliances,Medium-sized Retailer,35805,26598.0,8954.0,0.744025,19761.0,0.120281,17848.0
2,300157,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.000000,Haley's Flooring & Interiors (Inc),Home Improvement,Single Retailer,35805,26598.0,8954.0,0.744025,19761.0,0.120281,17848.0
3,1015,83984.05,171957.92,7895392.69,5382410.69,0.0,24085.39,314974.76,-0.511601,"STANTONS APPLIANCE SALES AND SERVICE, INC",Appliances,Medium-sized Retailer,70815,50862.0,10629.0,0.370214,30124.0,0.154859,26062.0
4,2594,121405.18,423310.06,0.00,0.00,0.0,0.00,0.00,-0.713200,ShoppersChoice.com,Appliances,Single Retailer,70815,50862.0,10629.0,0.370214,30124.0,0.154859,26062.0


,nmg_id,t12_volume,p12_volume,t12_credit_card_processing_total,t12_inventory_finance_total,t12_lease_to_own_total,t12_product_protection_total,t12_retail_credit_total,volume_growth,company_name,industry_name,number_of_locations,zip_code,median_income,households,poverty_rate,total_pop,vacant_housing_rate,income_per_capita
0,1000,171197.83,277735.60,0.00,1027568.42,0.0,195782.19,96131.18,-0.383594,A-1 HOME APPLIANCE CENTER,Appliances,Medium-sized Retailer,70058,49551.0,13844.0,0.514230,40191.0,0.081841,27419.0
1,1002,36097.56,89740.93,0.00,2050776.48,0.0,66616.09,178371.82,-0.597758,BOB WALLACE APPLIANCE,Appliances,Medium-sized Retailer,35805,26598.0,8954.0,0.744025,19761.0,0.120281,17848.0
2,300157,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.000000,Haley's Flooring & Interiors (Inc),Home Improvement,Single Retailer,35805,26598.0,8954.0,0.744025,19761.0,0.120281,17848.0
3,1015,83984.05,171957.92,7895392.69,5382410.69,0.0,24085.39,314974.76,-0.511601,"STANTONS APPLIANCE SALES AND SERVICE, INC",Appliances,Medium-sized Retailer,70815,50862.0,10629.0,0.370214,30124.0,0.154859,26062.0
4,2594,121405.18,423310.06,0.00,0.00,0.0,0.00,0.00,-0.713200,ShoppersChoice.com,Appliances,Single Retailer,70815,50862.0,10629.0,0.370214,30124.0,0.154859,26062.0


### Initial Basic KNN Model


In [33]:
from sklearn.compose import ColumnTransformer, make_column_selector
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.neighbors import NearestNeighbors
import pandas as pd
import numpy as np

def find_top_growing_neighbors(df, input_nmg_id, n_neighbors=5000, top_n=10, volume_weight=10):
    if 'nmg_id' not in df.columns:
        raise KeyError("'nmg_id' column not found in DataFrame.")
    df['nmg_id'] = df['nmg_id'].astype(str)

    # Convert specified categorical columns to strings
    categorical_features = ['industry_name', 'number_of_locations']
    for feature in categorical_features:
        df[feature] = df[feature].astype(str)

    numeric_features = ['p12_volume', 'median_income', 'households', 'poverty_rate', 'total_pop', 'vacant_housing_rate', 'income_per_capita']
    features_to_use = numeric_features + categorical_features

    # Adjusting preprocessing pipelines
    numeric_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='mean')),
        ('scaler', StandardScaler())])

    categorical_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
        ('encoder', OneHotEncoder(handle_unknown='ignore'))])

    # Preprocessing for numeric and categorical data
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', numeric_transformer, numeric_features),
            ('cat', categorical_transformer, categorical_features)])

    # Apply preprocessing
    X = preprocessor.fit_transform(df[features_to_use])

    # Adjust the weight of t12_volume
    t12_volume_index = features_to_use.index('p12_volume')  # Get the correct column index for t12_volume
    X[:, t12_volume_index] *= volume_weight

    # Fit Nearest Neighbors
    nn = NearestNeighbors(n_neighbors=n_neighbors)
    nn.fit(X)

    input_index = df.index[df['nmg_id'] == input_nmg_id].tolist()
    if not input_index:
        raise ValueError(f"No company found with nmg_id: {input_nmg_id}")

    # Find nearest neighbors
    distances, indices = nn.kneighbors([X[input_index[0]]])
    nearest_indices = indices[0][1:]  # Exclude the first index since it's the input itself

    neighbors_df = df.iloc[nearest_indices].copy()
    top_growing_neighbors = neighbors_df.sort_values(by='volume_growth', ascending=False).head(top_n)

    columns_to_display = ['nmg_id', 'company_name', 't12_volume', 'industry_name', 'number_of_locations', 'median_income', 'total_pop', 'volume_growth']
    return top_growing_neighbors[columns_to_display].reset_index(drop=True)


In [35]:
find_top_growing_neighbors(function_data, '8310')

,nmg_id,company_name,t12_volume,industry_name,number_of_locations,median_income,total_pop,volume_growth
0,106739,BILL GRAY & SONS,8073.78,Appliances,Single Retailer,50747.0,19441.0,0.497993
1,2187,GRAHAMS TV & APPLIANCE,4010.00,Appliances,Single Retailer,76730.0,19612.0,0.496269
2,50930,EZ Furniture Appliances Electronics,3050.00,Appliances,Large Retailer,61201.0,37411.0,0.484185
3,8914,Boerner's Appliance,699.00,Appliances,Single Retailer,58381.0,41574.0,0.477801
4,113174,APPLIANCE CENTER OF NORTHERN COLORADO,1686.00,Appliances,Single Retailer,67080.0,37656.0,0.436116
5,2263,BRIDGE CITY TV AND APPLIANCE,5335.11,Appliances,Single Retailer,45113.0,29613.0,0.429175
6,113762,APPLIANCES & MORE,3675.00,Appliances,Single Retailer,30832.0,30733.0,0.369735
7,50901,Texas App Saver LLC,7235.00,Bedding,Medium-sized Retailer,52961.0,29904.0,0.365094
8,2787,MOSES TRUE VALUE HARDWARE,2737.00,Appliances,Single Retailer,52208.0,28024.0,0.307068
9,7292,CHARLESTON APPLIANCE CENTER,899.00,Appliances,Single Retailer,46272.0,23642.0,0.306686


,nmg_id,company_name,t12_volume,industry_name,number_of_locations,median_income,total_pop,volume_growth
0,106739,BILL GRAY & SONS,8073.78,Appliances,Single Retailer,50747.0,19441.0,0.497993
1,2187,GRAHAMS TV & APPLIANCE,4010.00,Appliances,Single Retailer,76730.0,19612.0,0.496269
2,50930,EZ Furniture Appliances Electronics,3050.00,Appliances,Large Retailer,61201.0,37411.0,0.484185
3,8914,Boerner's Appliance,699.00,Appliances,Single Retailer,58381.0,41574.0,0.477801
4,113174,APPLIANCE CENTER OF NORTHERN COLORADO,1686.00,Appliances,Single Retailer,67080.0,37656.0,0.436116
5,2263,BRIDGE CITY TV AND APPLIANCE,5335.11,Appliances,Single Retailer,45113.0,29613.0,0.429175
6,113762,APPLIANCES & MORE,3675.00,Appliances,Single Retailer,30832.0,30733.0,0.369735
7,50901,Texas App Saver LLC,7235.00,Bedding,Medium-sized Retailer,52961.0,29904.0,0.365094
8,2787,MOSES TRUE VALUE HARDWARE,2737.00,Appliances,Single Retailer,52208.0,28024.0,0.307068
9,7292,CHARLESTON APPLIANCE CENTER,899.00,Appliances,Single Retailer,46272.0,23642.0,0.306686


In [36]:
columns_to_keep = [
    'nmg_id',
    'company_name',
    't12_volume',
    'industry_name',
    'number_of_locations',
    'median_income',
    'total_pop',
    'volume_growth']


filtered_data = function_data[function_data['nmg_id'] == "8310"][columns_to_keep]

# the initial nmg_id
filtered_data


,nmg_id,company_name,t12_volume,industry_name,number_of_locations,median_income,total_pop,volume_growth
500,8310,Hometown Furniture Center Inc.,0.0,Bedding,Single Retailer,54967.0,28475.0,0.0


,nmg_id,company_name,t12_volume,industry_name,number_of_locations,median_income,total_pop,volume_growth
500,8310,Hometown Furniture Center Inc.,0.0,Bedding,Single Retailer,54967.0,28475.0,0.0


### Understanding Product Usage of Comparable Retailers

In [37]:
from sklearn.compose import ColumnTransformer, make_column_selector
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.neighbors import NearestNeighbors
import pandas as pd
import numpy as np

def evaluate_product_usage(df):
    product_columns = [
        't12_credit_card_processing_total',
        't12_inventory_finance_total',
        't12_lease_to_own_total',
        't12_product_protection_total',
        't12_retail_credit_total'
    ]
    usage_columns = [
        'credit_card_processing',
        'inventory_finance',
        'lease_to_own',
        'product_protection',
        'retail_credit'
    ]
    for usage_col in usage_columns:
        df[usage_col] = 'No'
    for product_col, usage_col in zip(product_columns, usage_columns):
        df[usage_col] = df[product_col].apply(lambda x: 'Yes' if x > 0 else 'No')
    reordered_columns = ['nmg_id']
    for product_col, usage_col in zip(product_columns, usage_columns):
        reordered_columns.extend([usage_col, product_col])
    additional_columns = [col for col in df.columns if col not in reordered_columns and col not in product_columns]
    reordered_columns.extend(additional_columns)
    return df[reordered_columns]

def find_top_growing_neighbors_comp(df, input_nmg_id, n_neighbors=5000, top_n=10, volume_weight=10):
    if 'nmg_id' not in df.columns:
        raise KeyError("'nmg_id' column not found in DataFrame.")
    df['nmg_id'] = df['nmg_id'].astype(str)

    # cat to string
    categorical_features = ['industry_name', 'number_of_locations']
    for feature in categorical_features:
        df[feature] = df[feature].astype(str)

    numeric_features = ['p12_volume', 'median_income', 'households', 'poverty_rate', 'total_pop', 'vacant_housing_rate', 'income_per_capita']
    features_to_use = numeric_features + categorical_features

    #  preprocessing pipelines
    numeric_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='mean')),
        ('scaler', StandardScaler())])

    categorical_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
        ('encoder', OneHotEncoder(handle_unknown='ignore'))])

    preprocessor = ColumnTransformer(
        transformers=[
            ('num', numeric_transformer, numeric_features),
            ('cat', categorical_transformer, categorical_features)])

    #  apply
    X = preprocessor.fit_transform(df[features_to_use])

    # adjsuting the weight of t12_volume so that volumes are more comprable
    t12_volume_index = features_to_use.index('p12_volume')
    X[:, t12_volume_index] *= volume_weight

    # fitting KNN model
    nn = NearestNeighbors(n_neighbors=n_neighbors)
    nn.fit(X)

    input_index = df.index[df['nmg_id'] == input_nmg_id].tolist()
    if not input_index:
        raise ValueError(f"No company found with nmg_id: {input_nmg_id}")

    # Find nearest neighbors
    distances, indices = nn.kneighbors([X[input_index[0]]])
    nearest_indices = indices[0][1:]  # Exclude the first index since it's the input itself

    neighbors_df = df.iloc[nearest_indices].copy()
    top_growing_neighbors = neighbors_df.sort_values(by='volume_growth', ascending=False).head(top_n)

    # apply product usage evaluation to the top growing comprable retailers
    top_growing_neighbors_with_product_usage = evaluate_product_usage(top_growing_neighbors)

    # colums
    columns_to_display = [
        'nmg_id',
        'credit_card_processing',
        't12_credit_card_processing_total',
        'inventory_finance',
        't12_inventory_finance_total',
        'lease_to_own',
        't12_lease_to_own_total',
        'product_protection',
        't12_product_protection_total',
        'retail_credit',
        't12_retail_credit_total'
    ]

    return top_growing_neighbors_with_product_usage[columns_to_display].reset_index(drop=True)


In [39]:
top_neighbors_with_product_usage = find_top_growing_neighbors_comp(function_data, "8310")

top_neighbors_with_product_usage.head(5)

,nmg_id,credit_card_processing,t12_credit_card_processing_total,inventory_finance,t12_inventory_finance_total,lease_to_own,t12_lease_to_own_total,product_protection,t12_product_protection_total,retail_credit,t12_retail_credit_total
0,106739,No,0.0,Yes,51851.34,Yes,1100.00,Yes,477.39,Yes,132050.50
1,2187,No,0.0,Yes,134857.50,No,0.00,No,0.00,Yes,4617.00
2,50930,No,0.0,No,0.00,Yes,1703242.29,No,0.00,Yes,76139.23
3,8914,No,0.0,No,0.00,No,0.00,No,0.00,Yes,80884.28
4,113174,No,0.0,Yes,89631.80,No,0.00,No,0.00,No,0.00


,nmg_id,credit_card_processing,t12_credit_card_processing_total,inventory_finance,t12_inventory_finance_total,lease_to_own,t12_lease_to_own_total,product_protection,t12_product_protection_total,retail_credit,t12_retail_credit_total
0,106739,No,0.0,Yes,51851.34,Yes,1100.00,Yes,477.39,Yes,132050.50
1,2187,No,0.0,Yes,134857.50,No,0.00,No,0.00,Yes,4617.00
2,50930,No,0.0,No,0.00,Yes,1703242.29,No,0.00,Yes,76139.23
3,8914,No,0.0,No,0.00,No,0.00,No,0.00,Yes,80884.28
4,113174,No,0.0,Yes,89631.80,No,0.00,No,0.00,No,0.00
